In [1]:
import re
import csv
import time
import nltk
import json
import string
import numpy as np
import pandas as pd
from nltk import stem
import sklearn.metrics
from random import randint
from numpy.linalg import norm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from scipy.optimize import minimize


with open('../CollaborativeFiltering/new_userDict.json', 'r') as fp:
    
    new_userDict = json.load(fp)
    
    
with open('../CollaborativeFiltering/new_isbnDict.json', 'r') as fp:
    
    new_isbnDict = json.load(fp)

    
    
with open('../CollaborativeFiltering/dict_row.json', 'r') as fp:
    
    dict_row = json.load(fp)
    
    
with open('../CollaborativeFiltering/dict_col.json', 'r') as fp:
    
    dict_col = json.load(fp)
    
        
print("Ok")

Ok


In [3]:
with open('../CollaborativeFiltering/clustering/CLUSTERS_ITEMS.json', 'r') as fp:
    
    CLUSTERS_ITEMS = json.load(fp)
    
    
with open('../CollaborativeFiltering/clustering/CLUSTERS_USERS.json', 'r') as fp:
    
    CLUSTERS_USERS = json.load(fp)

    
with open('../CollaborativeFiltering/clustering/file/clusters_dict_row.json', 'r') as fp:
    
    clusters_dict_row = json.load(fp)
       
with open('../CollaborativeFiltering/clustering/file/clusters_dict_col.json', 'r') as fp:
    
    clusters_dict_col = json.load(fp)
    
print("Ok") 

with open('../CollaborativeFiltering/clustering/index_book_user_clusters/index_user_cluster.json', 'r') as fp:
    
    index_user_cluster = json.load(fp)
    
    
with open('../CollaborativeFiltering/clustering/index_book_user_clusters/index_book_cluster.json', 'r') as fp:
    
    index_book_cluster = json.load(fp)
    
print("Ok")

Ok
Ok


# SVD

In [6]:
def createSampleDict(new_isbnDict, new_userDict, t1, t2):
    

    small_isbnDict = {}

    for book in new_isbnDict:

        temp = new_isbnDict[book]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t1:

            small_isbnDict[book] = new_isbnDict[book]   

    small_userDict = {}

    for user in new_userDict:

        temp = new_userDict[user]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t2:

            small_userDict[user] = new_userDict[user]
            
            
    return small_isbnDict, small_userDict

In [25]:
def computeMatrices(train_userDict,train_isbnDict,small_userDict,small_isbnDict, dict_row, dict_col):

    n = len(small_isbnDict)
    m = len(small_userDict)
    
    index = sorted(small_userDict.keys())
    columns = sorted(small_isbnDict.keys())

    dict_row = {k:v for v,k in enumerate(index)}
    dict_col = {k:v for v,k in enumerate(columns)}

    u = np.zeros((m,n)) 
    R = np.zeros((m,n))
    for user in train_userDict:
        for isbn in train_userDict[user]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_userDict[user][isbn]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue

    for isbn in train_isbnDict:
        for user in train_isbnDict[isbn]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_isbnDict[isbn][user]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue
                
    small_utility_DataFrame = pd.DataFrame(u, index = index, columns = columns)
    #R = pd.DataFrame(R, index = index, columns = columns)

    return u, R, small_utility_DataFrame

In [10]:
u_cluster, R_cluster, utility_DataFrame_cluster = computeMatrices(CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                           CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                            clusters_dict_row,clusters_dict_col)


u, R, utility_DataFrame = computeMatrices(new_userDict,new_isbnDict,new_userDict,new_isbnDict,dict_row,dict_col)

# normalize values X and theta

In [11]:
def normalizeUtility(U, R):

    U_i_mean = np.zeros((U.shape[0]))
    U_j_mean = np.zeros((U.shape[1]))

    for i in range(U.shape[0]):

        index = np.where(R[i,:]==1)[0]

        U_i_mean[i] = U[i,index].mean()
        U[i,index] = U[i,index] - U[i,index].mean()
        
        
    for j in range(U.shape[1]):

        index = np.where(R[:,j]==1)[0]

        U_j_mean[j] = U[index,j].mean()
        U[index,j] = U[index,j] - U[index,j].mean()
        
    U = np.nan_to_num(U)
    U_i_mean = np.nan_to_num(U_i_mean)
    U_j_mean = np.nan_to_num(U_j_mean)
        
    return U, U_i_mean, U_j_mean

# regularized cost function

$$ J = \sum_{i=0}^{n}\sum_{j=0}^{m}[(\sum_{k=1}^{k} X_{ik}\theta_{kj}) - U_{ij}]^2  + \lambda \sum_{i=0}^{n}\sum_{k=0}^{k}X_{ik}^2  +  \lambda \sum_{j=0}^{m}\sum_{k=0}^{k}\theta_{kj}^2 $$


# regularized gradient


$$\dfrac{\partial J}{\partial X_{ik}}  =  \sum_{j=0}^{m}\{[(\sum_{k=1}^{k} X_{ik}\theta_{kj}) - U_{ij}]\theta_{kj}\} + \lambda X_{ik} $$

$$\dfrac{\partial J}{\partial \theta_{kj}}  =  \sum_{i=0}^{n}\{[(\sum_{k=1}^{k} X_{ik}\theta_{kj}) - U_{ij}]X_{ik}\} + \lambda \theta_{kj} $$

In [12]:
def gradFunction(c,X,theta,alpha):
    
    grad_X = np.dot(c,theta.T) + alpha*X
    grad_theta = np.dot(X.T,c) + alpha*theta
    
    return grad_X, grad_theta


def costFunction(parameters,U,R,t,alpha):
    
    n, m = U.shape
    
    X = np.reshape(parameters[:n*t], (n,t))
    theta = np.reshape(parameters[n*t:], (t,m))
    
    c = np.dot(X,theta)
    c = c*R - U 
    C = (c**2).sum()
    J = C + alpha*((X**2).sum()) + alpha*((theta**2).sum())
    
    grad_X, grad_theta = gradFunction(c,X,theta,alpha)
    grad = np.concatenate((np.ravel(grad_X), np.ravel(grad_theta)))
    
    return J, grad  

In [13]:
def Xtheta(parameters,U,t):
    
    n, m = U.shape
    
    X = np.reshape(parameters[:n*t], (n,t))
    theta = np.reshape(parameters[n*t:], (t,m))
    
    return X,theta
    

In [14]:
def prediction(x, U, t, U_i_mean, U_j_mean):
    
    X_new,theta_new = Xtheta(x,U,t)
    U_new = np.dot(X_new,theta_new)
    
    for row in range(U_new.shape[0]):
        
        U_new[row,:] = U_new[row,:] + U_i_mean[row]
        
    for col in range(U_new.shape[1]):
        
        U_new[:,col] = U_new[:,col] + U_j_mean[col]
        
    U_new[U_new<0] = 0
    U_new[U_new>10] = 10
    U_new = U_new.astype(int)
    
    return U_new

In [49]:
def rmseFunction(U_new,u,R):
    
    index = [u[R==1]!=0]
    
    
    s = np.sqrt((((U_new[R==1][index] - u[R==1][index])**2).sum())/len(u[R==1][index]))
    
    return s

In [50]:
rmseFunction(U_new,u,R)

0.90368162335382007

In [16]:
def mainSVD(u,R,t,alpha):
    
    X = np.random.random((u.shape[0],t))
    theta = np.random.random((t,u.shape[1]))
    
    U, U_i_mean, U_j_mean = normalizeUtility(u.copy(),R)
    
    parameters = np.concatenate((np.ravel(X), np.ravel(theta)))
    
    fmin = minimize(fun=costFunction, x0=parameters, args=(U, R,t,alpha), 
                method='CG', jac=True, options={'maxiter': 100})
    
    U_new = prediction(fmin.x, U, t, U_i_mean, U_j_mean)
    
    return U_new

In [18]:
small_isbnDict, small_userDict = createSampleDict(new_isbnDict, new_userDict, 20, 20)

index = sorted(small_userDict.keys())
columns = sorted(small_isbnDict.keys())

dict_row = {k:v for v,k in enumerate(index)}
dict_col = {k:v for v,k in enumerate(columns)}

#utility_DataFrame = pd.DataFrame(u, index = index, columns = columns)
#R_DataFrame = pd.DataFrame(R, index = index, columns = columns)

In [29]:
def main(train_userDict,train_isbnDict,small_userDict,small_isbnDict,dict_row,dict_col,t=100,alpha=10):
    
    u,R,utility_DataFrame = computeMatrices(train_userDict,train_isbnDict,small_userDict,small_isbnDict, dict_row, dict_col)
    U_new = mainSVD(u,R,t,alpha)
    rmse = rmseFunction(U_new,u,R)
    
    return rmse, U_new, u, R

In [30]:
rmse_tot, U_new_tot, u_tot, R_tot = main(small_userDict,small_isbnDict,small_userDict,small_isbnDict,dict_row,dict_col,100,1)

/home/georgos/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/georgos/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [51]:
RMSE = []
k = 5
t = 100
alpha = 1
list_user = np.array(sorted(small_userDict.keys()))
b = len(small_userDict)//k

for iterator in range(k):
    
    train_userDict, train_isbnDict, test = selectSample(iterator,b,list_user,small_userDict, small_isbnDict)
    
    u, R, utility_DataFrame = computeMatrices(train_userDict,train_isbnDict,small_userDict,small_isbnDict, dict_row, dict_col)
    U_new = mainSVD(u,R,t,alpha)
    
    R_f = reduced(test,dict_row,R_tot)
    rmse_test = rmseFunction(U_new,u_tot,R_f)
    print("Ok")
    
    
    RMSE.append(rmse_test)

/home/georgos/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/georgos/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/georgos/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 3521 but corresponding boolean dimension is 704
  app.launch_new_instance()
/home/georgos/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 3521 but corresponding boolean dimension is 704


Ok
Ok
Ok
Ok
Ok


In [52]:
np.mean(RMSE)

2.3153658697085371

In [21]:
def selectSample(i,b,list_user,small_userDict, small_isbnDict):

    test_index = [k for k in range((i*b),(i+1)*b)]
    train_index = [k for k in range(len(list_user)) if k not in test_index]

    train = list_user[train_index]
    test  = list_user[test_index]

    train_userDict = {}
    train_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in train:

        train_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                train_userDict[user]
                train_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
    return train_userDict, train_isbnDict, test


def reduced(test,dict_row,R):
    
    v = []
    for row in test:
        v.append(dict_row[row])

    v = sorted(v)    
    R_f = R[v,:]
    
    return R_f

In [104]:
test_index = np.random.choice(indices, size = b, replace=False)

In [120]:
indices = [i for i in range(len(list_user))]
indices = np.array(indices)
k = 5
b = len(small_userDict)//k

def selectSampleRandom(indices,b,list_user,small_userDict, small_isbnDict):
    
    test_index = np.random.choice(indices, size = b, replace=False)
    
    indices = np.delete(indices,test_index)
    
    train_index = [k for k in range(len(list_user)) if k not in test_index]
    

    train = list_user[train_index]
    test  = list_user[test_index]

    train_userDict = {}
    train_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in train:

        train_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                train_userDict[user]
                train_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
    return train_userDict, train_isbnDict, test, indices

In [121]:
for i in range(5):
    
    train_userDict, train_isbnDict, test, indices = selectSampleRandom(indices,b,list_user,small_userDict, small_isbnDict)
    

3521
2817
2265
1806
1425
